In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from smodels.decomposition.theorySMS import TheorySMS
from smodels.experiment.expSMS import ExpSMS
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.experiment.defaultFinalStates import finalStates
from smodels.base.model import Model
from collections import OrderedDict
from smodels.base.physicsUnits import fb, GeV
from itertools import product
from smodels.experiment.databaseObj import Database
import copy
from smodels.experiment.expAuxiliaryFuncs import bracketToProcessStr
import re

### Get database

In [2]:
db = Database('official')

In [3]:
def convertAxesStr(inStr):
    entryDict = {}
    newStr = inStr[:]
    iel = 0
    for x in re.split(r',\s*(?![^()]*\))', inStr):
        x = x.replace(']','').replace('[','')
        entryDict["X%i" %iel] = x
        newStr = newStr.replace(x,"X%i" %iel,1)
        iel += 1
    axes = eval(newStr,entryDict)
    
    return axes

def getAxesSMS(txAxes):
    """
    Generates a dummy SMS with nodes holding axis labels.
    """
    axesSMS = []
    if isinstance(txAxes,str):
        txAxes = [txAxes]
    for txAx in txAxes:
        axis = convertAxesStr(txAx)
        axisStr = str([[['anySM'] for ptc in br[:-1]] for br in axis])
        finalStates = [str(br[-1]) for br in axis]
        intermediateStates = [[str(ptc) for ptc in br[:-1]] for br in axis]
        axSMS = ExpSMS.from_string(axisStr,finalState=finalStates,
                                intermediateState=intermediateStates)
        axesSMS.append(axSMS)
    
    return axesSMS

In [4]:
stop = False
for exp in db.getExpResults():
    for tx in exp.getTxNames():
        if not tx.axes:
            continue
        try:
            sms = getAxesSMS(tx.axes)
        except:
            print(exp.globalInfo.id)
            print(tx)
            stop = True
        if stop: break
    if stop: break

ATLAS-SUSY-2016-06
TDTM1F


In [5]:
print(tx,tx.axes)

TDTM1F ['[[(x, y), x - 1.0], [(x, y), x - 1.0]]', ' [[(x, y), x], [(x, y), x]]']


In [7]:
print(tx.axes[0])
print(convertAxesStr(tx.axes[0]))
print(getAxesSMS(tx.axes[0]))

[[(x, y), x - 1.0], [(x, y), x - 1.0]]
[['(x, y)', 'x - 1.0'], ['(x, y)', 'x - 1.0']]


IndexError: list index out of range

In [ ]:
s = '[(x,y),x-1.0],[(x,y),x-10.0]'
print(s[s.find("(")+1:s.find(")")])

In [ ]:
# inStr = '[[x, 0.5*x + 0.5*y, y], [x, 0.5*x + 0.5*y, y]]'
inStr = '[[(x,y),x-1.0], [(x,y),x-10.0], [x,y,z]]'

# currentStr = inStr[:].replace(' ','')
# currentStr = currentStr[1:-1]
# branches = currentStr.split('],[')
# print(branches)